<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/3%D0%9A%D0%B5%D0%B9%D1%811_%D0%9F%D0%BE%D0%B8%D1%81%D0%BA_%D0%B2%D1%80%D0%B5%D0%B4%D0%BE%D0%BD%D0%BE%D1%81%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BA%D0%BE%D0%B4%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import ast # Для анализа деревьев синтаксического разбора кода на Python
import importlib # Для динамического импорта модулей
import sys
import inspect  # Для исследования объектов во время выполнения (кода, функций, классов, модулей)
FORBIDDEN_NAMES = ['exec', 'eval'] # Запрещенные элементы по условию задания

def check_for_malicious_code(code, output_file):
    try:
        # Чтение кода из файла
        with open(f"{code}.py", 'r', encoding='utf-8') as f:
            source_code = f.read()

    except SyntaxError as e:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"Код содержит синтаксическую ошибку: {type(e).__name__}")
        return False
    except Exception as e:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"Код содержит вредоносные элементы: {type(e).__name__}")
        return False

    try:
        tree = ast.parse(source_code) # Cоздается объект-дерево для синтаксического разбора кода

        for i in ast.walk(tree): # Обход дерева
            if isinstance(i, ast.Attribute): # Перебор атрибутов объектов
                if i.attr in FORBIDDEN_NAMES: # Поиск запрещенных элементов в атрибуте объекта
                    with open(output_file, 'w', encoding='utf-8') as f:
                        f.write(f"Код содержит запрещенный атрибут: {id.attr}")
                    return False

            if isinstance(i, ast.Call): # Проверка: является ли узел вызовом функции/метода
                if isinstance(i.func, ast.Name) and i.func.id == 'eval': # Вызываемый объект - простое имя (не атрибут объекта) и он eval
                    if not i.args: # eval() без аргументов по условию задания
                        with open(output_file, 'w', encoding='utf-8') as f:
                            f.write("Код содержит вызов eval()")
                        return False
            return True

    except Exception as e:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"Код содержит вредоносные элементы: {type(e).__name__}")
        return False

def run_tests(standard_code, user_code, test_cases, output_file):
    # Сначала проверяем пользовательский код на вредоносные элементы
    security_check_result = check_for_malicious_code(user_code, "security_check.txt")

    if not security_check_result:
        # Читаем ошибку безопасности
        with open("security_check.txt", 'r', encoding='utf-8') as f:
            security_error = f.read()

        # Записываем ошибку в основной файл с нужным форматом
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("Ошибка при проверке модуля user_malicious:\n")
            f.write(security_error)
        return

    # Поиск модулей (файлов .py) в директории
    sys.path.insert(0, '') if '' not in sys.path else None

    try:
        # Импорт модулей
        standard_module = importlib.import_module(standard_code)
        user_module = importlib.import_module(user_code)

        # Выполнение функций из модулей и получения результатов
        s_res = start_module(standard_module, test_cases)
        u_res = start_module(user_module, test_cases)

        # Запись результатов сравнения работы функций в файл
        with open(output_file, 'w', encoding='utf-8') as f:
            # Сравниваем результаты каждого теста отдельно
            if isinstance(s_res, list) and isinstance(u_res, list):
                # Тестируем каждый случай
                for i, (s, u) in enumerate(zip(s_res, u_res), 1):
                    if s == u:
                        f.write(f'Тест пройден\n')
                    else:
                        f.write(f'Тест не пройден\n')
                        f.write(f'Ожидаемый результат: {s}\n')
                        f.write(f'Результат пользователя: {u}\n')
            else:
                # Если не список - один кортеж
                if s_res == u_res:
                    f.write('Тест пройден')
                else:
                    f.write('Тест не пройден\n')
                    f.write(f'Ожидаемый результат: {s_res}\n')
                    f.write(f'Результат пользователя: {u_res}')
    except ModuleNotFoundError as e:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"Ошибка импорта модуля: {e}")

# Функция для запуска функций из модуля
def start_module(name_module, test_cases):
    # Находим первую вызываемую функцию - делаем перебор всех атрибутов модуля
    for name in dir(name_module):
        # Отфильтровываем приватные и специальные методы и проверяем, что объект можно вызвать как функцию
        if not name.startswith('_') and callable(getattr(name_module, name)):
            try:
                if test_cases:
                    # Список для хранения всех результатов
                    res = [
                        getattr(name_module, name)(*args) if isinstance(args, (list, tuple))
                        else getattr(name_module, name)(args)
                        for args in test_cases
                    ]
                    return res  # Возвращаем список всех результатов
                else:
                    return getattr(name_module, name)()  # Вызов функции без аргументов
            except Exception as e:
                print(f"Ошибка при выполнении кода пользователя: {e}")
                return None
    return None



run_tests('etalon', 'user_malicious', [(1, 2), (3, 4), (5, 6)], 'output2.txt')




